# Getting Started with the Agent Development Kit (ADK) with Neo4j

This notebook shows how to use the Vertex AI Agent Framework to setup a multi-agent system integrating Neo4j.

The Vertex AI ADK is a python-based SDK that empowers developers to build multi-agent applications with custom logic, tools, and integrations.

In this notebook you will learn how to do the following:

*  Install Agent Development Kit
*  Define functions as tools
*  Define a database and analyst agent
*  Create a "Bom & Supplier Research" Multi-Agent


## Learn more about Neo4j

* https://neo4j.com/genai
* https://neo4j.com/developer
* https://graphrag.com
* https://neo4j.com/labs/genai-ecosystem
* [Neo4j and MCP Toolbox](https://neo4j.com/blog/developer/ai-agents-gen-ai-toolbox/)

## Learn more about the ADK

* [Launch Blog](https://cloud.google.com/blog/products/ai-machine-learning/build-and-manage-multi-system-agents-with-vertex-ai)
* [Developer Blog](https://developers.googleblog.com/en/agent-development-kit-easy-to-build-multi-agent-applications/)
* https://pypi.org/project/google-adk/
* https://google.github.io/adk-docs/
* https://github.com/google/adk-samples/
* [LLM Agents](https://google.github.io/adk-docs/agents/llm-agents/#putting-it-together-example)
* [Deploy to Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/)

# Install the Agent Development Kit

In [1]:
# from google.colab import auth
# auth.authenticate_user()
!gcloud auth application-default login > /dev/null 2>&1

In [6]:
%pip install --upgrade google-cloud-aiplatform[adk,agent_engines] neo4j-rust-ext python-dotenv

Note: you may need to restart the kernel to use updated packages.


# Developer API or Vertex AI

The ADK supports two APIs:

* Google Gemini API (AI Studio)
* Vertex AI Gemini API.

In [1]:
from dotenv import load_dotenv
import getpass
import os

load_dotenv('nb.env', override=True)

if not os.environ.get('NEO4J_URI'):
    os.environ['NEO4J_URI'] = getpass.getpass('NEO4J_URI:\n')
if not os.environ.get('NEO4J_USERNAME'):
    os.environ['NEO4J_USERNAME'] = getpass.getpass('NEO4J_USERNAME:\n')
if not os.environ.get('NEO4J_PASSWORD'):
    os.environ['NEO4J_PASSWORD'] = getpass.getpass('NEO4J_PASSWORD:\n')

NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [2]:
# Only run this block for ML Developer API. Use your own API key.
import os

GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "0"


In [9]:
# Only run this block for Vertex AI API Use your own project / location.
# import os
#
# GOOGLE_CLOUD_PROJECT = "neo4jeventdemos" #@param {type:"string"}
#
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "1"
# os.environ["GOOGLE_CLOUD_PROJECT"] = GOOGLE_CLOUD_PROJECT
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"

# Neo4j Database Connectivity

We define a small class `neo4jDatabase` that manages connecting to Neo4j and running read queries.

In [3]:
import logging

logger = logging.getLogger('agent_neo4j_cypher')
logger.info("Initializing Database for tools")

import logging

logging.getLogger("neo4j").setLevel(logging.ERROR)
logging.getLogger("google_genai").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="google_genai.types")
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j.notifications")


In [4]:
from neo4j import GraphDatabase
from typing import Any
import re

class neo4jDatabase:
    def __init__(self,  neo4j_uri: str, neo4j_username: str, neo4j_password: str):
        """Initialize connection to the Neo4j database"""
        logger.debug(f"Initializing database connection to {neo4j_uri}")
        d = GraphDatabase.driver(neo4j_uri, auth=(neo4j_username, neo4j_password))
        d.verify_connectivity()
        self.driver = d

    def is_write_query(self, query: str) -> bool:
      return re.search(r"\b(MERGE|CREATE|SET|DELETE|REMOVE|ADD)\b", query, re.IGNORECASE) is not None

    #TODO: Use result transformer here to r: r.data()
    def _execute_query(self, query: str, params: dict[str, Any] | None = None) -> list[dict[str, Any]]:
        """Execute a Cypher query and return results as a list of dictionaries"""
        logger.debug(f"Executing query: {query}")
        try:
            if self.is_write_query(query):
                logger.error(f"Write query not supported {query}")
                raise "Write Queries are not supported in this agent"
                # logger.debug(f"Write query affected {counters}")
                # result = self.driver.execute_query(query, params)
                # counters = vars(result.summary.counters)
                # return [counters]
            else:
                #TODO: Add Routing here for effciency
                result = self.driver.execute_query(query, params)
                results = [dict(r) for r in result.records]
                logger.debug(f"Read query returned {len(results)} rows")
                return results
        except Exception as e:
            logger.error(f"Database error executing query: {e}\n{query}")
            raise

In [5]:
db = neo4jDatabase(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

In [6]:
# Testing database connection
db._execute_query("RETURN 1")

[{'1': 1}]

## Defining Functions for our Database Agent

* get_schema - Retrieve Database Schema for the LLM
* execute_read_query - Execute Cypher Read Query

In [7]:
def get_schema() -> list[dict[str,Any]]:
  """Get the schema of the database, returns node-types(labels) with their types and attributes and relationships between node-labels
  Args: None
  Returns:
    list[dict[str,Any]]: A list of dictionaries representing the schema of the database
    For example
    ```
    [{'label': 'Person','attributes': {'summary': 'STRING','id': 'STRING unique indexed', 'name': 'STRING indexed'},
      'relationships': {'HAS_PARENT': 'Person', 'HAS_CHILD': 'Person'}}]
    ```
  """
  try:
      results = db._execute_query(
              """
call apoc.meta.data() yield label, property, type, other, unique, index, elementType
where elementType = 'node' and not label starts with '_'
with label,
collect(case when type <> 'RELATIONSHIP' then [property, type + case when unique then " unique" else "" end + case when index then " indexed" else "" end] end) as attributes,
collect(case when type = 'RELATIONSHIP' then [property, head(other)] end) as relationships
RETURN label, apoc.map.fromPairs(attributes) as attributes, apoc.map.fromPairs(relationships) as relationships
              """
          )
      return results
  except Exception as e:
      return [{"error":str(e)}]

In [8]:
get_schema()

[{'label': 'Supplier',
  'attributes': {'annual_spend': 'FLOAT',
   'tier': 'STRING',
   'sub_type': 'STRING',
   'code': 'STRING indexed'},
  'relationships': {'LOCATED_AT': 'GeoLocation'}},
 {'label': 'Item',
  'attributes': {'desc_embedding': 'LIST',
   'description': 'STRING',
   'family': 'STRING',
   'is_finished_product': 'BOOLEAN',
   'name': 'STRING',
   'sku_id': 'STRING indexed'},
  'relationships': {'AT': 'Supplier', 'BOM': 'Item', 'DEPENDS_ON': 'Country'}},
 {'label': 'Customer',
  'attributes': {'annual_revenue': 'FLOAT', 'code': 'INTEGER indexed'},
  'relationships': {'LOCATED_AT': 'GeoLocation'}},
 {'label': 'GeoLocation',
  'attributes': {'longitude': 'FLOAT',
   'latitude': 'FLOAT',
   'geo_point': 'POINT indexed'},
  'relationships': {}},
 {'label': 'Country',
  'attributes': {'name': 'STRING', 'code': 'STRING indexed'},
  'relationships': {}}]

In [9]:
from typing import Optional


def execute_read_query(query: str, params: Optional[dict[str, Any]]=None) -> list[dict[str, Any]]:
    """
    Execute a Neo4j Cypher query and return results as a list of dictionaries
    Args:
        query (str): The Cypher query to execute
        params (dict[str, Any], optional): The parameters to pass to the query or None.
    Raises:
        Exception: If there is an error executing the query
    Returns:
        list[dict[str, Any]]: A list of dictionaries representing the query results
    """
    try:
        if params is None:
            params = {}
        results = db._execute_query(query, params)
        return results
    except Exception as e:
        return [{"error":str(e)}]

In [10]:
execute_read_query("RETURN 1", None)

[{'1': 1}]

## Defining Functions for our Investment Research Agent

* get_schema - Retrieve Database Schema for the LLM
* get_investors - Returns the investor in the company with this name or id.

In [42]:
def get_finished_product_families() -> list[dict[str, Any]]:
    """
    Get the different type of Product Families - for finished products only.
    Use this when asked for different types, families, or groups of finished products.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item)
        WHERE i.is_finished_product
        RETURN i.family AS family, count(*) AS numberOfProducts
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_finished_product_families()

[{'family': 'FarmTractor', 'numberOfProducts': 21521},
 {'family': 'SeedPlanter', 'numberOfProducts': 21488},
 {'family': 'FieldSprayer', 'numberOfProducts': 21519},
 {'family': 'CropHarvester', 'numberOfProducts': 21406},
 {'family': 'HayCollector', 'numberOfProducts': 21671},
 {'family': 'CabWiringUnit', 'numberOfProducts': 17},
 {'family': 'ControlAssembly', 'numberOfProducts': 9},
 {'family': 'MachineRig', 'numberOfProducts': 14},
 {'family': 'HydraulicSystem', 'numberOfProducts': 3},
 {'family': 'ChassisFrame', 'numberOfProducts': 9},
 {'family': 'DrivePlatform', 'numberOfProducts': 10}]

In [43]:
def get_component_families() -> list[dict[str, Any]]:
    """
    Get the different type of Component Families - for components only -not finished products.
    Use this when asked for different types, families or groups of components and/or parts.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item)
        WHERE NOT i.is_finished_product
        RETURN i.family AS family, count(*) AS numberOfComponents
        """)
        return results
    except Exception as e:
        return [{"error":str(e)}]
#test
get_component_families()

[{'family': 'MachineRig', 'numberOfComponents': 20017},
 {'family': 'DrivePlatform', 'numberOfComponents': 19896},
 {'family': 'HydraulicSystem', 'numberOfComponents': 20192},
 {'family': 'ChassisFrame', 'numberOfComponents': 19893},
 {'family': 'ControlAssembly', 'numberOfComponents': 19824},
 {'family': 'CabWiringUnit', 'numberOfComponents': 19886},
 {'family': 'Bushing', 'numberOfComponents': 162},
 {'family': 'Spring', 'numberOfComponents': 144},
 {'family': 'GearSet', 'numberOfComponents': 169},
 {'family': 'ElectricalControlBox', 'numberOfComponents': 77},
 {'family': 'ComponentModule', 'numberOfComponents': 63},
 {'family': 'Board', 'numberOfComponents': 82},
 {'family': 'SteeringColumn', 'numberOfComponents': 65},
 {'family': 'FrameSegment', 'numberOfComponents': 62},
 {'family': 'SensorModule', 'numberOfComponents': 67},
 {'family': 'OperatorCab', 'numberOfComponents': 12},
 {'family': 'AxleAssembly', 'numberOfComponents': 79},
 {'family': 'ControlUnit', 'numberOfComponents': 

In [44]:
def get_item(sku_id: str) -> dict[str, Any]:
    """
    Gets Item properties
    Args:
        sku_id (str): code uniquely identifying the item which can be a component or finished product
    Returns:
        dict[str, Any]: item properties
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: "35472245A"})
        RETURN i.sku_id AS sku_id, i.name AS name, i.family AS family, i.is_finished_product AS is_finished_product
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return {"error": str(e)}

In [45]:
get_item("35472245A")

[{'sku_id': '35472245A',
  'name': 'FarmTractor_7YUC0',
  'family': 'FarmTractor',
  'is_finished_product': True}]

In [46]:
def get_known_item_country_dependencies(sku_id: str) -> list[dict[str, Any]]:
    """
    Retrieve all known country dependencies for a given item, which can represent either a finished product, a component, or both. This includes data on sourcing, manufacturing, or other relationships tied to specific countries.
    Dependencies are determined recursively by examining all connected components in the item's bill of materials (BOM). For each component, this function identifies:
    1. **Known Country Dependencies:** A list of countries and their corresponding dependency descriptions (e.g., sourcing steel, manufacturing).
    2. **Supply Chain Dependencies:** A visual representation of how components with such country dependencies are linked to the provided item through potentially multiple stages of the BOM hierarchy, represented as a sequence of components connected by arrows, culminating in the provided item.

    Note: This function only retrieves available data, which may be incomplete due to limited internal intelligence.

    Args:
        sku_id (str): code uniquely identifying the component or finished product
    Returns:
        list[dict[str, Any]]: a list of components in this items bill of materials with country dependencies along with their country dependencies and how they relate to the item in the bill of materials hierarchy.
    """
    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        MATCH path=(i)<-[:BOM*]-(comp:Item)-[r:DEPENDS_ON]->(c:Country)
        RETURN
        comp.sku_id AS sku_id,
        comp.name AS name,
        collect({country: c.name, dependency: r.description}) AS known_country_dependencies,
        ' <- ' + apoc.text.join([n IN tail(nodes(path))[..-1] | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return [{"error": str(e)}]

In [47]:
get_known_item_country_dependencies("35472245A")

[{'sku_id': 'M7307U91X',
  'name': 'RawWire_YCBEO',
  'known_country_dependencies': [{'dependency': 'sourcing raw copper cathode',
    'country': 'Chile'},
   {'dependency': 'drawing and annealing facilities', 'country': 'Mexico'},
   {'dependency': 'drawing and annealing facilities', 'country': 'Viet Nam'},
   {'dependency': 'sourcing raw copper cathode', 'country': 'Peru'}],
  'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35523080X (CabWiringUnit_UJ3OE) <- M7307U91X (RawWire_YCBEO)'},
 {'sku_id': 'M7307U92X',
  'name': 'PrecisionBolt_O12J1',
  'known_country_dependencies': [{'dependency': 'final machining and threading processes',
    'country': 'United States of America'},
   {'dependency': 'sourcing raw titanium sponge', 'country': 'Japan'},
   {'dependency': 'procuring alloying elements (aluminum and vanadium)',
    'country': 'Russian Federation'}],
  'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35523087X (CabWiringUnit_CZ0N2) <- M7307U92X

In [48]:
def get_item_bill_of_materials(sku_id: str) -> list[dict[str, Any]]:
    """
    Retrieve the complete Bill of Materials (BOM) hierarchy for a given finished product or component identified by the SKU.
    This function provides a detailed view of all components involved in the production of the specified item.
    It identifies components recursively linked to the given item through the BOM hierarchy.

    Key Features:
    1. **Bill of Material Dependencies:** Identifies all components recursively linked to the given item through the BOM hierarchy.
    2. **Dependency Path Representation:** Produces a flattened tree structure where each entry represents a sequence of components
       connected by arrows, showing the hierarchical relationships from the base components to the provided item.


    Args:
        sku_id (str): The SKU (Stock Keeping Unit) that uniquely identifies a product, component, or part.

    Returns:
        list[dict[str, Any]]: A list of dependency paths represented as strings. Each path shows the components in the BOM hierarchy
        leading to the specified item, connected by arrows (` <- `). For example:
            " <- 35472245 (CabWiringUnit_Z5OO3)"
            " <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O)"
            " <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O) <- M3571002BX (CabWiringUnit_A9FH4)"
    """

    try:
        results = db._execute_query("""
        MATCH (i:Item {sku_id: $sku_id})
        MATCH path=(i)<-[:BOM*]-(:Item)
        RETURN ' <- ' + apoc.text.join([n IN tail(nodes(path)) | n.sku_id + ' (' + coalesce(n.name, '') + ')' ], ' <- ') AS supply_chain_dependency
        """, {"sku_id": sku_id})
        return results
    except Exception as e:
        return [{"error": str(e)}]


In [49]:
get_item_bill_of_materials("35472245A")

[{'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6495003X (DrivePlatform_SBK8U)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499003X (MachineRig_RFYOE)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499002X (HydraulicSystem_EE22P)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35370826 (MachineRig_INWWQ)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35433612 (ChassisFrame_2GN2W)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475030 (ElectricalControlBox_6ROWJ)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 28028534X (ControlAssembly_JRO9M)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O)'},
 {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475154X (CabWiringUnit_VT75O) <- M357

In [ ]:
#search for dependencies on a

#1 get country options - separate function
#2 search for countries and dependencies
#come dependencies

# Define our Agents

An AI agent reasons, plans, and takes actions.

The agent takes actions via access to **tools**, deciding how and when to invoke a tool. The agent also manages orchestration, creating a plan for answering a user query and adapting to responses that aren't quite correct.

Agent **tools** can be Python functions, or Vertex AI Extensions, or MCP Servers.

## Creating the Database Agent.

* display name
* instructions - detailed, making it clear exactly how the agent should behave and which tools to use when
* tools



In [50]:
from google.adk.agents import Agent

In [51]:
MODEL="gemini-2.5-pro-preview-03-25"

In [52]:
database_agent = Agent(
    model=MODEL,
    name='graph_database_agent',
    instruction="""
      You are an Neo4j graph database and Cypher query expert, that must use the database schema with a user question and repeatedly generate valid cypher statements
      to execute on the database and answer the user's questions in a friendly manner in natural language.
      If in doubt the database schema is always prioritized when it comes to nodes-types (labels) or relationship-types or property names, never take the user's input at face value.
      If the user requests also render tables, charts or other artifacts with the query results.
      Always validate the correct node-labels at the end of a relationship based on the schema.

      If a query fails or doesn't return data, use the error response 3 times to try to fix the generated query and re-run it, don't return the error to the user.
      If you cannot fix the query, explain the issue to the user and apologize.

      Fetch the graph database schema first and keep it in session memory to access later for query generation.
      Keep results of previous executions in session memory and access if needed, for instance ids or other attributes of nodes to find them again
      removing the need to ask the user. This also allows for generating shorter, more focused and less error-prone queries
      to for drill downs, sequences and loops.
      If possible resolve names to primary keys or ids and use those for looking up entities.
      The schema always indicates *outgoing* relationship-types from an entity to another entity, the graph patterns read like english language.
      `company has supplier` would be the pattern `(o:Organization)-[:HAS_SUPPLIER]->(s:Organization)`

      To get the schema of a database use the `get_schema` tool without parameters. Store the response of the schema tool in session context
      to access later for query generation.

      To answer a user question generate one or more Cypher statements based on the database schema and the parts of the user question.
      If necessary resolve categorical attributes (like names, countries, industries, publications) first by retrieving them for a set of entities to translate from the user's request.
      Use the `execute_query` tool repeatedly with the Cypher statements, you MUST generate statements that use named query parameters with `$parameter` style names
      and MUST pass them as a second dictionary parameter to the tool, even if empty.
      Parameter data can come from the users requests, prior query results or additional lookup queries.
      After the data for the question has been sufficiently retrieved, pass the data and control back to the parent agent.
    """,
    tools=[
        get_schema, execute_read_query
    ]
)

In [53]:
bom_supplier_research_agent = Agent(
    model=MODEL,
    name='bom_supplier_research_agent',
    instruction="""
    You are an agent that has access to a database of bill of materials (BOM), supplier, product/component, and customer relationships.
    Use the provided tools to answer questions. 
    when returning information, try to always return not just the factual attribute data but also
    codes, skus, and ids to allow the other agents to investigate them more.
    """,
    tools=[
        get_schema, get_item, get_known_item_country_dependencies, get_item_bill_of_materials, get_finished_product_families, get_component_families
    ]
)

In [54]:
root_agent = Agent(
    model=MODEL,
    name='bom_supplier_agent',
    global_instruction = "",
    instruction="""
    You are an agent that has access to a database of bill of materials (BOM), supplier, product/component, and customer relationships.
    You have a set of agents to retrieve information, you should prefer the research agents over the database agent - particularly for questions around bill of materials, component & product info, and country dependencies. Only use the database agent if you have to as a fallback.
    If the user requests it, do render tables, charts or other artifacts with the research results.
    """,

    sub_agents=[bom_supplier_research_agent, database_agent]
)

# Let's try it

You can [run the ADK locally](https://google.github.io/adk-docs/get-started/local-testing/#expected-output) a web application with `adk web` or the FastAPI server with `adk api_server`.

You can also deploy the agent to [Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/) or to [Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/).

In [55]:
APP_NAME = 'BOM & Supplier Analyst'
USER_ID = 'Zach Blumenfeld'

from google.adk.runners import InMemoryRunner
from google.genai.types import Part, UserContent


runner = InMemoryRunner(app_name=APP_NAME, agent=root_agent)

session = runner.session_service.create_session( app_name=runner.app_name, user_id=USER_ID)

async def run_prompt(new_message: str):
  content = UserContent(parts=[Part(text=new_message)])
# print (content)
  result = None
  async for event in runner.run_async(user_id=session.user_id, session_id=session.id, new_message=content):
#    print(event.content.model_dump(exclude_none=True))
#    print(event.content.parts)
    for part in event.content.parts:
      print(part.text, part.function_call, part.function_response)
      if part.text:
#        print(part.text)
        result = part.text
  return result

In [56]:
from IPython.display import Markdown, display

res = await run_prompt('what types of finished products are there?')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-c6dc17f0-2839-4557-b66e-909ab7c71ba0' args={'agent_name': 'bom_supplier_research_agent'} name='transfer_to_agent' None
None None id='adk-c6dc17f0-2839-4557-b66e-909ab7c71ba0' name='transfer_to_agent' response={}
None id='adk-b1a51069-a143-405b-b59e-e05e8fcb7fbc' args={} name='get_finished_product_families' None
None None id='adk-b1a51069-a143-405b-b59e-e05e8fcb7fbc' name='get_finished_product_families' response={'result': [{'family': 'FarmTractor', 'numberOfProducts': 21521}, {'family': 'SeedPlanter', 'numberOfProducts': 21488}, {'family': 'FieldSprayer', 'numberOfProducts': 21519}, {'family': 'CropHarvester', 'numberOfProducts': 21406}, {'family': 'HayCollector', 'numberOfProducts': 21671}, {'family': 'CabWiringUnit', 'numberOfProducts': 17}, {'family': 'ControlAssembly', 'numberOfProducts': 9}, {'family': 'MachineRig', 'numberOfProducts': 14}, {'family': 'HydraulicSystem', 'numberOfProducts': 3}, {'family': 'ChassisFrame', 'numberOfProducts': 9}, {'family': 'DrivePlatfor

Based on the available data, here are the families of finished products:
*   FarmTractor (21521 products)
*   SeedPlanter (21488 products)
*   FieldSprayer (21519 products)
*   CropHarvester (21406 products)
*   HayCollector (21671 products)
*   CabWiringUnit (17 products)
*   ControlAssembly (9 products)
*   MachineRig (14 products)
*   HydraulicSystem (3 products)
*   ChassisFrame (9 products)
*   DrivePlatform (10 products)

In [57]:
res = await run_prompt('Can you summerize the BOM and potential country dependencies for product "35472245A"')
print("\n\n\n\nFinal Response:")
display(Markdown(res))

None id='adk-195671ba-c9c4-4f8b-a3fc-6c6eb2596fb7' args={'sku_id': '35472245A'} name='get_item_bill_of_materials' None
None id='adk-4a82cb90-7f43-4ae8-934c-88a141d81071' args={'sku_id': '35472245A'} name='get_known_item_country_dependencies' None
None None id='adk-195671ba-c9c4-4f8b-a3fc-6c6eb2596fb7' name='get_item_bill_of_materials' response={'result': [{'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6495003X (DrivePlatform_SBK8U)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499003X (MachineRig_RFYOE)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- M6499002X (HydraulicSystem_EE22P)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35370826 (MachineRig_INWWQ)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35433612 (ChassisFrame_2GN2W)'}, {'supply_chain_dependency': ' <- 35472245 (CabWiringUnit_Z5OO3) <- 35475

Okay, here's a summary of the Bill of Materials (BOM) and potential country dependencies for product SKU "35472245A":

**Bill of Materials (BOM) Summary:**

The product "35472245A" appears to be primarily built upon the component `35472245 (CabWiringUnit_Z5OO3)`. This main component integrates numerous sub-components and assemblies, including various types of:
*   Cab Wiring Units (e.g., `35475154X`, `16031437X`, `9396190X`, `35523080X`, `35523087X`)
*   Machine Rigs (e.g., `M6499003X`, `35370826`, `35472500`, `35473640X`)
*   Chassis Frames (e.g., `35433612`, `35464217`, `35670669X`, `33225099X`)
*   Hydraulic Systems (e.g., `M6499002X`, `9396196X`, `16087891X`)
*   Drive Platforms (e.g., `M6495003X`, `9396276X`, `16087850X`)
*   Control Assemblies (e.g., `28028534X`, `9396465X`, `33131665X`)
*   An Electrical Control Box (`35475030`)

Some components are nested deeper in the hierarchy. For instance:
*   `35523080X (CabWiringUnit_UJ3OE)` uses `M7307U91X (RawWire_YCBEO)`
*   `35523087X (CabWiringUnit_CZ0N2)` uses `M7307U92X (PrecisionBolt_O12J1)`
*   `35523089X (DrivePlatform_ZB6KE)` uses `M7307U93X (Fastener_CP82S)`

**Potential Country Dependencies:**

Based on the known data, the following components within the BOM for "35472245A" have country-specific dependencies:

1.  **Component:** `M7307U91X (RawWire_YCBEO)`
    *   **Supply Chain Path:** `<- 35472245 (CabWiringUnit_Z5OO3) <- 35523080X (CabWiringUnit_UJ3OE) <- M7307U91X (RawWire_YCBEO)`
    *   **Known Dependencies:**
        *   Sourcing raw copper cathode: Chile, Peru
        *   Drawing and annealing facilities: Mexico, Viet Nam

2.  **Component:** `M7307U92X (PrecisionBolt_O12J1)`
    *   **Supply Chain Path:** `<- 35472245 (CabWiringUnit_Z5OO3) <- 35523087X (CabWiringUnit_CZ0N2) <- M7307U92X (PrecisionBolt_O12J1)`
    *   **Known Dependencies:**
        *   Sourcing raw titanium sponge: Japan
        *   Procuring alloying elements (aluminum and vanadium): Russian Federation
        *   Final machining and threading processes: United States of America

3.  **Component:** `M7307U93X (Fastener_CP82S)`
    *   **Supply Chain Path:** `<- 35472245 (CabWiringUnit_Z5OO3) <- 35523089X (DrivePlatform_ZB6KE) <- M7307U93X (Fastener_CP82S)`
    *   **Known Dependencies:**
        *   Sourcing raw titanium: Russian Federation
        *   Precision forging: Germany

This summary highlights the key sub-components and identifies specific lower-level parts (`RawWire_YCBEO`, `PrecisionBolt_O12J1`, `Fastener_CP82S`) with dependencies related to sourcing raw materials (copper, titanium, alloying elements) and manufacturing processes (drawing, annealing, machining, forging) in countries like Chile, Peru, Mexico, Viet Nam, Japan, Russia, the USA, and Germany.

In [64]:
for session in runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID).sessions:
  print(session.model_dump())

{'id': '6bb051de-50d7-4739-ac26-be45b6ed51c4', 'app_name': 'BOM & Supplier Analyst', 'user_id': 'Zach Blumenfeld', 'state': {}, 'events': [], 'last_update_time': 1745199538.265297}


In [67]:
result = await run_prompt("Summarize the results of the previous research questions")
print("\n\n\n\nFinal Response:")
display(Markdown(result))

In [66]:
for session in runner.session_service.list_sessions(app_name=APP_NAME, user_id=USER_ID).sessions:
  print(f"Deleting session {session}")
  runner.session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=session.id)

Deleting session id='6bb051de-50d7-4739-ac26-be45b6ed51c4' app_name='BOM & Supplier Analyst' user_id='Zach Blumenfeld' state={} events=[] last_update_time=1745199634.542561
